# Análise Comparativa


Essa etapa tem como objetivo comparar alguns modelos para encontrar um que possa ser melhor utilizado dentro do problema, para isso antes realizamos a preparação e o pré-processamento dos dados.

## Preparação dos dados

In [1]:
from IPython.display import Markdown
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate, ShuffleSplit, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB

In [9]:
# Ler o csv
df_day = pd.read_csv('day.csv')

# Ler o dicionário de dados
dicionario_dados_day = pd.read_csv('dicionario_dados_day.csv')

### Tratamento dos dados

Aqui realizamos a normalização, codificação e o tratamento de dados discrepantes e/ou faltantes dentro do conjunto de dados

## Metodologia

"A equipe deve utilizar um método de validação cruzada (holdout, k-fold, Monte Carlo) apresentado em sala de aula;
 A equipe deve apresentar no mínimo quatro modelos: um baseline, para servir de comparação a pelo menos outros três (ou mais) experimentos com modelos de famílias diferentes. Deverão ser utilizadas pelo menos duas métricas para efetuar a comparação entre os modelos"

## Configuração do experimento

- Separar o conjunto em teste e treino
- Realizar teste com os modelos definidos

## Resultados e discussão

- Expor em uma tabela a comparação dos modelos para explicar qual deles melhor se encaixa para o projeto